# AoC Day 13 Challenge. First section is grabbing the input text for the day

In [2]:
# Common Imports
import pycurl as curl
from io import BytesIO
import pandas as pd
import numpy as np
import re
import os
from functools import partial
from copy import deepcopy
import matplotlib.pyplot as plt
import sys
from tqdm import tqdm
from collections import defaultdict
sys.getrecursionlimit()
sys.setrecursionlimit(100_000)

In [57]:
# Get input as `input`
year = 2024
day = 13

with open('/home/alec/aoc_cookie.txt', 'r') as file:
    cookie = file.read() 

buffer = BytesIO()
c = curl.Curl()
c.setopt(curl.URL, f"https://adventofcode.com/{year}/day/{day}/input")
c.setopt(curl.COOKIE, f"session={cookie}")
c.setopt(curl.WRITEDATA, buffer)
c.perform()
c.close()

input = buffer.getvalue().decode('utf-8')

In [40]:
input ="""Button A: X+94, Y+34
Button B: X+22, Y+67
Prize: X=8400, Y=5400

Button A: X+26, Y+66
Button B: X+67, Y+21
Prize: X=12748, Y=12176

Button A: X+17, Y+86
Button B: X+84, Y+37
Prize: X=7870, Y=6450

Button A: X+69, Y+23
Button B: X+27, Y+71
Prize: X=18641, Y=10279"""

# Part 1

In [58]:
entries = input.splitlines()

rules = defaultdict(list)
id = 0
total = 0
# Get rules
for entry in entries:
    if len(entry) == 0:
        id += 1
        continue
    if "Button A:" in entry:
        _a = entry.split("Button A:")[1]
        _x = _a.split("X")[1]
        x = int(_x.split(", ")[0])
        _y = _a.split("Y")[1]
        y = int(_y.split(", ")[0])
        rules[id].append((x,y))
    elif "Button B:" in entry:
        _a = entry.split("Button B:")[1]
        _x = _a.split("X")[1]
        x = int(_x.split(", ")[0])
        _y = _a.split("Y")[1]
        y = int(_y.split(", ")[0])
        rules[id].append((x,y))
    else:
        _a = entry.split("Prize:")[1]
        x = int(_a.split("X=")[1].split()[0][:-1])
        y = int(_a.split("Y=")[1].split()[0])
        rules[id].append((x,y))


In [61]:
# Part 1
# THis sounds like a gcf problem
# Or this is probably a graph problem
# like A*
a_cost = 3
b_cost = 1

def add(p1, p2):
    tmp = []
    for _p1, _p2 in zip(p1,p2):
        tmp.append(_p1 + _p2)
    return tuple(tmp)

def mult(p1, m):
    tmp = []
    for _p1 in p1:
        tmp.append(_p1*m)
    return tuple(tmp)
epsilon = 0.01
total = 0
scale = 10000000000000
for k, rule in rules.items():
    # print(rule)
    m_numer = (rule[2][1] + scale) - (rule[0][1]*((rule[2][0] + scale)/rule[0][0]))
    m_denom = rule[1][1] - rule[1][0]*(rule[0][1]/rule[0][0])
    m = m_numer/m_denom

    n = (rule[2][0] + scale - rule[1][0]*(np.round(m))) / rule[0][0]

    # print(m,n, a_cost*n + b_cost*m)
    if n <= 0 or abs(np.round(n) - n) > epsilon:# or n > 100:
        # print("n",n, "m",m)
        continue
    if m <= 0 or abs(np.round(m) - m) > epsilon:# or m > 100:
        # print("n",n,"m",m)
        continue
    # print(n,m)
    total += a_cost*np.round(n) + b_cost*np.round(m)
print(f"{total:20f}")

77204516023437.000000


In [6]:
"""
| a_x | * N + | b_x | * M = | prize_x |
| a_y |       | b_y |       | prize_y |

min(N*3 + M)

a_x * N + b_x * M = p_x
a_y * N + b_y * M = p_y

a_y * (p_x - b_x * M)/a_x + b_y * M = p_y

(a_y * p_x/a_x) + M * [(-b_x*a_y/a_x) + b_y] = p_y

numer = p_y - a_y * p_x / a_x
denom = b_y - b_x*a_y/a_x

M = numer / denom
N = (prize_x - b_x * M) / a_x
"""

"""
| a_x | * N + | b_x | * M = | prize_x + SCALE |
| a_y |       | b_y |       | prize_y + SCALE|

min(N*3 + M)

a_x * N + b_x * M = p_x
a_y * N + b_y * M = p_y

a_y * (p_x - b_x * M)/a_x + b_y * M = p_y

(a_y * p_x/a_x) + M * [(-b_x*a_y/a_x) + b_y] = p_y

numer = p_y - a_y * p_x / a_x
denom = b_y - b_x*a_y/a_x

3038.29787 + M * 66.7659574 = 5400
"""

'\n| a_x | * N + | b_x | * M = | prize_x |\n| a_y |       | b_y |       | prize_y |\n\nmin(N*3 + M)\n\na_x * N + b_x * M = p_x\na_y * N + b_y * M = p_y\n\na_y * (p_x - b_x * M)/a_x + b_y * M = p_y\n\n(a_y * p_x/a_x) + M * [(-b_x*a_y/a_x) + b_y] = p_y\n\nnumer = p_y - a_y * p_x / a_x\ndenom = b_y - b_x*a_y/a_x\n\n3038.29787 + M * 66.7659574 = 5400\n'

In [51]:
max_float = np.finfo(np.float64).max

print(max_float)

1.7976931348623157e+308
